# 시계열 데이터의 처리
pandas에서는 여러 가지 시계열 데이터를 처리하는 기능을 가지고 있습니다.
### 주가 취득하기
시계열 데이터의 샘플을 가져와보도록 하겠습니다.

In [2]:
import pandas as pd
anime_stock_price_csv = './sample-data/anime/anime_stock_price.csv'
df = pd.read_csv(anime_stock_price_csv, index_col=0, parse_dates=['Date'])
df.head()

,TOEI ANIMATION,IG Port
Date,,
2015-01-01,3356.86,1201.51
2015-01-02,3356.86,1201.51
2015-01-05,3396.12,1218.44
2015-01-06,3361.77,1201.51
2015-01-07,3297.97,1202.51


#### 시계열 데이터로 사용된 함수
pct_change() 메서드를 사용하여 직전의 데이터 값으로부터 변화율을 산출해보겠습니다.

In [3]:
pd.options.display.max_rows = 10 # pandas로 실행하는 행 수를 지정
pct_change = df['TOEI ANIMATION'].pct_change()
pct_change

Date
2015-01-01         NaN
2015-01-02    0.000000
2015-01-05    0.011695
2015-01-06   -0.010114
2015-01-07   -0.018978
                ...   
2016-12-26    0.001725
2016-12-27    0.010326
2016-12-28    0.015333
2016-12-29    0.013422
2016-12-30    0.000000
Name: TOEI ANIMATION, Length: 522, dtype: float64

cumprod를 이용하여 누적값을 산출할 수도 있습니다. 첫날의 주가를 1로 해서 주가의 누적 리턴을 산출해보겠습
니다.

In [4]:
cumulative_returns = (pct_change + 1).cumprod()
cumulative_returns[0] = 1
cumulative_returns

Date
2015-01-01    1.000000
2015-01-02    1.000000
2015-01-05    1.011695
2015-01-06    1.001463
2015-01-07    0.982457
                ...   
2016-12-26    1.722833
2016-12-27    1.740624
2016-12-28    1.767312
2016-12-29    1.791034
2016-12-30    1.791034
Name: TOEI ANIMATION, Length: 522, dtype: float64

rolling을 이용하여 데이터 범위를 이동시키면서 함수를 적용할 수 있습니다. 제 1인수에 구간을 정수로 지정하여
이 구간에 대해 함수를 적용해보겠습니다. 5일간의 이동 평균을 산출해보겠습니다. 과거 5일간의 평균을 산출하
기 위해 최소 4일간은 NaN이 됩니다.

In [5]:
df['TOEI ANIMATION'].rolling(5).mean()

Date
2015-01-01         NaN
2015-01-02         NaN
2015-01-05         NaN
2015-01-06         NaN
2015-01-07    3353.916
                ...   
2016-12-26    5793.260
2016-12-27    5799.232
2016-12-28    5821.132
2016-12-29    5868.912
2016-12-30    5916.692
Name: TOEI ANIMATION, Length: 522, dtype: float64

위에서는 mean() 함수를 사용했지만, 임의의 함수를 적용할 수 있습니다. 30일간의 변동성을 산출하는 함수를 작성해서 적용해보겠습니다.

In [7]:
import numpy as np
def historical_volatility(x):
    logreturns = np.diff(np.log(x)) # 로그 수익률
    return np.sqrt(365 * logreturns.var())
df['TOEI ANIMATION'].rolling(30).apply(historical_volatility)

C:\Users\USER\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """


Date
2015-01-01         NaN
2015-01-02         NaN
2015-01-05         NaN
2015-01-06         NaN
2015-01-07         NaN
                ...   
2016-12-26    0.210959
2016-12-27    0.208397
2016-12-28    0.192327
2016-12-29    0.196373
2016-12-30    0.178677
Name: TOEI ANIMATION, Length: 522, dtype: float64

### DatetimeIndex
pandas의 DatetimeIndex는 datetime형 처리에 특화된 Index입니다. pandas.date_range()함수는 지정한 주기(표준 설정 1일)의 DatetimeIndex를 작성합니다. 2017년 1월 1일부터 2017년 2월 1일까지의 기간에 대하여 1시간 단위로 DatetimeIndex를 작성해보겠습니다.

In [8]:
ix =pd.date_range('2017-01', '2017-02', freq = '1H')
ix

DatetimeIndex(['2017-01-01 00:00:00', '2017-01-01 01:00:00',
               '2017-01-01 02:00:00', '2017-01-01 03:00:00',
               '2017-01-01 04:00:00', '2017-01-01 05:00:00',
               '2017-01-01 06:00:00', '2017-01-01 07:00:00',
               '2017-01-01 08:00:00', '2017-01-01 09:00:00',
               ...
               '2017-01-31 15:00:00', '2017-01-31 16:00:00',
               '2017-01-31 17:00:00', '2017-01-31 18:00:00',
               '2017-01-31 19:00:00', '2017-01-31 20:00:00',
               '2017-01-31 21:00:00', '2017-01-31 22:00:00',
               '2017-01-31 23:00:00', '2017-02-01 00:00:00'],
              dtype='datetime64[ns]', length=745, freq='H')

#### Series의 인덱스로 사용하는 경우
DatetimeIndex는 Series나 DataFrame의 인덱스로 사용할 수 있습니다.

In [9]:
time_series = pd.Series(np.arange(len(ix)), index = ix)
time_series

2017-01-01 00:00:00      0
2017-01-01 01:00:00      1
2017-01-01 02:00:00      2
2017-01-01 03:00:00      3
2017-01-01 04:00:00      4
                      ... 
2017-01-31 20:00:00    740
2017-01-31 21:00:00    741
2017-01-31 22:00:00    742
2017-01-31 23:00:00    743
2017-02-01 00:00:00    744
Freq: H, Length: 745, dtype: int32

### 시계열 데이터를 추출하기
DatetimeIndex의 인덱서에는 datetime형과 문자열형 양쪽을 지정할 수 있습니다. datetime형으로 데이터를 선택
해보겠습니다.

In [10]:
from datetime import datetime
df.loc[datetime(2016, 1, 4)]

TOEI ANIMATION    5699.74
IG Port            822.66
Name: 2016-01-04 00:00:00, dtype: float64

#### 인덱서에 문자열을 지정하는 경우
인덱서에 문자열을 지정하는 경우에는 아래와 같이 지정하면 됩니다.

In [11]:
df.loc['2016-01-04']

TOEI ANIMATION    5699.74
IG Port            822.66
Name: 2016-01-04 00:00:00, dtype: float64

In [12]:
df.loc['Jan-04-2016']

TOEI ANIMATION    5699.74
IG Port            822.66
Name: 2016-01-04 00:00:00, dtype: float64

#### 특정 년도나 월의 데이터만 추출하는 경우
인덱스에 년, 월을 지정해서 특정 년, 월의 데이터만 추출할 수 있습니다. 2015년의 데이터만 추출해보겠습니다.

In [13]:
print(df.loc['2015'].head())

            TOEI ANIMATION  IG Port
Date                               
2015-01-01         3356.86  1201.51
2015-01-02         3356.86  1201.51
2015-01-05         3396.12  1218.44
2015-01-06         3361.77  1201.51
2015-01-07         3297.97  1202.51


2016년 5월의 데이터만 추출해보겠습니다.

In [14]:
print(df.loc['2016-05'].head())

            TOEI ANIMATION  IG Port
Date                               
2016-05-02         4703.29   933.34
2016-05-03         4703.29   933.34
2016-05-04         4703.29   933.34
2016-05-05         4703.29   933.34
2016-05-06         4678.41   953.28


#### 특정 년도, 월을 지정해서 슬라이스하는 경우
인덱스에 년, 월을 지정해서 특정 년, 월 기간의 데이터만 추출해보겠습니다.

In [15]:
print(df.loc['2015-12':'2016-01'])

            TOEI ANIMATION  IG Port
Date                               
2015-12-01         5947.13   910.41
2015-12-02         5917.44   896.45
2015-12-03         5917.44   893.46
2015-12-04         5867.97   888.47
2015-12-07         5917.44   892.46
...                    ...      ...
2016-01-25         5452.36   704.99
2016-01-26         5491.94   702.00
2016-01-27         5521.63   721.94
2016-01-28         5679.95   733.91
2016-01-29         5670.06   739.89

[44 rows x 2 columns]


#### 지정한 시각만의 데이터를 추출하는 경우
datetime.time 형으로 지정한 시각만의 데이터를 추출할 수도 있습니다. 9시 데이터만을 추출 해보도록 하겠습니다.

In [16]:
from datetime import time
time_series.loc[time(9,0)]

2017-01-01 09:00:00      9
2017-01-02 09:00:00     33
2017-01-03 09:00:00     57
2017-01-04 09:00:00     81
2017-01-05 09:00:00    105
                      ... 
2017-01-27 09:00:00    633
2017-01-28 09:00:00    657
2017-01-29 09:00:00    681
2017-01-30 09:00:00    705
2017-01-31 09:00:00    729
Freq: 24H, Length: 31, dtype: int32

#### 지정한 시간대만 추출하는 경우
between_time() 메서드를 이용하여 지정한 시간대만 추출할수도 있습니다. 9시부터 12시 사이의 데이터를 추출해보겠습니다.

In [17]:
time_series.between_time(time(9,0),time(12,0))

2017-01-01 09:00:00      9
2017-01-01 10:00:00     10
2017-01-01 11:00:00     11
2017-01-01 12:00:00     12
2017-01-02 09:00:00     33
                      ... 
2017-01-30 12:00:00    708
2017-01-31 09:00:00    729
2017-01-31 10:00:00    730
2017-01-31 11:00:00    731
2017-01-31 12:00:00    732
Length: 124, dtype: int32

### 리샘플링
resample() 메서드를 이용하여 시계열 데이터의 빈도를 변환할 수 있습니다. 일별 데이터를 주별이나 월별 등의 데
이터로 변환해보겠습니다.

In [18]:
df['TOEI ANIMATION'].resample('M').mean().head()

Date
2015-01-31    3647.080000
2015-02-28    3612.302500
2015-03-31    3625.770455
2015-04-30    3477.555455
2015-05-31    3653.990476
Freq: M, Name: TOEI ANIMATION, dtype: float64

ohlc() 메서드를 사용하여 4개 값(어느 일정 기간의 시작가, 마감가, 최고가, 최저가)으로 변환할 수 있습니다. 일별 데이터에서 주별 4개 값으로 변환해보겠습니다.

In [19]:
df['TOEI ANIMATION'].resample('W').ohlc().head()

,open,high,low,close
Date,,,,
2015-01-04,3356.86,3356.86,3356.86,3356.86
2015-01-11,3396.12,3513.90,3297.97,3513.90
2015-01-18,3513.90,3872.16,3435.38,3872.16
2015-01-25,3877.07,3877.07,3739.66,3739.66
2015-02-01,3774.01,3965.41,3774.01,3965.41
